In [1]:
import os
import pyodbc
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [2]:
path = r"C:\Users\jshernandezm\genommalabinternacional\Cristian Javier Sanchez Yepez - NoB2B\Alkosto"
name_file_last_week = '\\reportesCKPRnewfilesINF_PROVEEDORESVDR0014997_20210401-20210425_20210426MER02-01.csv' 
name_file_accumulated = '\\reportesCKPRnewfilesINF_PROVEEDORESVDR0014997_20210401-20210430_20210501-01.csv'

df_last = pd.read_csv(path + name_file_last_week, encoding='latin1')
df_accum = pd.read_csv(path + name_file_accumulated, encoding='latin1')

final_date = datetime.strptime(name_file_accumulated.split('_')[2].split('-')[1], '%Y%m%d')

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=yes;')

query_week = "SELECT TmpSemanaAnioGenomma FROM Gnm_DWH.dbo.Dim_Tiempo WHERE TmpFecha = '{0}'"

week = pd.read_sql(query_week.format(final_date.strftime('%Y-%m-%d')), conn)

In [4]:
def clean_columns(df):
    data = df.copy()
    try:
        data['EAN'] = data['EAN'].apply(lambda x: int(x.replace('=','').replace('"','')))
    except:
        pass
    data['ID'] = data['UNIDAD NEGOCIO INVENTARIO'] + data['EAN'].astype(str)
    return data

In [5]:
def deaccumlute(last_week, accumulat, date, week):
    last_week_trans = clean_columns(last_week)[['UNIDADES VENDIDAS', 'ID']]
    accumulat_trans = clean_columns(accumulat)[['UNIDAD NEGOCIO INVENTARIO', 'EAN', 'DESCRIPCIÓN', 'INVENTARIO TOTAL', 'UNIDADES VENDIDAS', 'PRECIO + IVA', 'ID']]
    join = accumulat_trans.merge(last_week_trans, on='ID', how='left')
    join['UNIDADES VENDIDAS_y'].fillna(0, inplace=True)
    join['Unidades'] = join['UNIDADES VENDIDAS_x'] - join['UNIDADES VENDIDAS_y']
    join['Anio'] = date.year
    join['Semana'] = week.loc[0].values[0]
    join['Fecha Inicio'] = date - timedelta(days=6)
    join['Fecha Fin'] = date
    join['Cliente'] = 'Alkosto'
    join.drop(['UNIDADES VENDIDAS_x', 'ID', 'UNIDADES VENDIDAS_y'], axis=1, inplace=True)
    join.rename({'UNIDAD NEGOCIO INVENTARIO':'Cod_Local', 'DESCRIPCIÓN':'Descripcion_Prod', 'INVENTARIO TOTAL':'Inventario', 'PRECIO + IVA':'Precio + IVA'}, axis=1, inplace=True)
    return join[['Anio', 'Semana', 'Fecha Inicio', 'Fecha Fin', 'Cliente', 'EAN', 'Descripcion_Prod', 'Cod_Local', 'Unidades', 'Inventario', 'Precio + IVA']]

In [6]:
final = deaccumlute(df_last, df_accum, final_date, week)
file_name = 'Alkosto' + '_' + str(final_date.year) + '_' + str(week.loc[0].values[0]) + '.csv'
print(file_name)
final.to_csv(path + '\\Layout\\' + file_name , index=False)

Alkosto_2021_18.csv


%time
path = r"C:\Users\jshernandezm\genommalabinternacional\Cristian Javier Sanchez Yepez - NoB2B\Alkosto"
files = os.listdir(path)[2:7]
for name_file_last_week, name_file_accumulated in zip(files[:4], files[1:]):
    df_last = pd.read_csv(path + '\\' + name_file_last_week, encoding='latin1')
    df_accum = pd.read_csv(path + '\\' + name_file_accumulated, encoding='latin1')
    final_date = datetime.strptime(name_file_accumulated.split('_')[2].split('-')[1], '%Y%m%d')
    week = pd.read_sql(query_week.format(final_date.strftime('%Y-%m-%d')), conn)
    final = deaccumlute(df_last, df_accum, final_date, week)
    file_name = 'Alkosto' + '_' + str(final_date.year) + '_' + str(week.loc[0].values[0]) + '.csv'
    final.to_csv(path + '\\Layout\\' + file_name , index=False)
    print(file_name, 'listones!')